In [1]:
# Импортируем необходимые для работы библиотеки

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
import re
import plotly.graph_objects as go
import math

from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import mean_squared_error


# убираем предупреждения
import warnings
warnings.filterwarnings('ignore')

На обработанных нами данных мы будем строить модель предсказания цены объекта недвижимости. Мы будем строить модель линейной регрессии с разбивками учебная-валидационная-тестовая выборки в соотношениях: 80-10-10 / 70-10-20 / 70-15-15.

Дакже построим K-fold валидацию с разбивкой на десять фрагментов. 

Метрику для оценки будем использовать - MSE.

In [11]:
# Загрузим данные из файла

data = pd.read_csv('Coding_dataset_03.csv')

data

,baths,sqft,PrivatePool,propertyType_Condo,propertyType_Condo/Townhome/Row Home/Co-Op,propertyType_High Rise,propertyType_Land,propertyType_Multi Family,propertyType_Other,propertyType_Single Family Home,...,PK–6_distance,PK–8_distance,K–6_distance,K–8_distance,K to 8_distance,other_distance,beds_sqft,stories_coding,target_log,State_coding
0,4,2900.0,0,0,0,0,0,0,0,1,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.03704,528.000,2,12.943237,1
1,3,1947.0,0,0,0,0,0,0,0,1,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00758,396.000,2,12.644328,3
2,2,3000.0,1,0,0,0,0,0,0,1,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00840,396.000,1,14.878496,4
3,8,6457.0,0,0,0,0,0,0,0,1,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00952,660.000,4,14.688894,1
4,3,1477.0,0,0,0,0,0,0,0,0,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00168,396.000,2,8.517193,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367038,6,4017.0,1,0,0,0,0,0,0,1,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00312,660.000,0,14.037854,2
367039,3,2000.0,0,1,0,0,0,0,0,0,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.25000,396.000,4,13.422466,2
367040,3,1152.0,0,0,0,0,0,0,0,1,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.02083,396.000,2,13.176852,4
367041,3,1990.0,0,0,0,0,0,0,1,0,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.07692,8.479,2,10.448715,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367043 entries, 0 to 367042
Data columns (total 81 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   baths                                       367043 non-null  int64  
 1   sqft                                        367043 non-null  float64
 2   PrivatePool                                 367043 non-null  int64  
 3   propertyType_Condo                          367043 non-null  int64  
 4   propertyType_Condo/Townhome/Row Home/Co-Op  367043 non-null  int64  
 5   propertyType_High Rise                      367043 non-null  int64  
 6   propertyType_Land                           367043 non-null  int64  
 7   propertyType_Multi Family                   367043 non-null  int64  
 8   propertyType_Other                          367043 non-null  int64  
 9   propertyType_Single Family Home             367043 non-null  int64  
 

In [12]:
# Построим модель линейной регрессии с разбивкой на тренировочную, валидационную и тестовую выборки 80 - 10 - 10

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Split data into training, validation, and testing sets
X_train, X_val_test, y_train, y_val_test = train_test_split(data.drop('target_log', axis=1), data['target_log'], test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the validation data
y_pred_val = model.predict(X_val)

# Evaluate the model on the validation data
mse_val = mean_squared_error(y_val, y_pred_val)
print(f'Validation MSE: {mse_val:.2f}')

# Make predictions on the testing data
y_pred_test = model.predict(X_test)

# Evaluate the model on the testing data
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Testing MSE: {mse_test:.2f}')

Validation MSE: 0.95
Testing MSE: 0.96


In [13]:

# Построим модель линейной регрессии с разбивкой на тренировочную, валидационную и тестовую выборки 70 - 10 - 20

# Split data into training, validation, and testing sets
X_train, X_val_test, y_train, y_val_test = train_test_split(data.drop('target_log', axis=1), data['target_log'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=2/3, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the validation data
y_pred_val = model.predict(X_val)

# Evaluate the model on the validation data
mse_val = mean_squared_error(y_val, y_pred_val)
print(f'Validation MSE: {mse_val:.2f}')

# Make predictions on the testing data
y_pred_test = model.predict(X_test)

# Evaluate the model on the testing data
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Testing MSE: {mse_test:.2f}')

Validation MSE: 0.96
Testing MSE: 0.96


In [14]:
# Построим модель линейной регрессии с разбивкой на тренировочную, валидационную и тестовую выборки 70 - 15 - 15

# Split data into training, validation, and testing sets
X_train, X_val_test, y_train, y_val_test = train_test_split(data.drop('target_log', axis=1), data['target_log'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the validation data
y_pred_val = model.predict(X_val)

# Evaluate the model on the validation data
mse_val = mean_squared_error(y_val, y_pred_val)
print(f'Validation MSE: {mse_val:.2f}')

# Make predictions on the testing data
y_pred_test = model.predict(X_test)

# Evaluate the model on the testing data
mse_test = mean_squared_error(y_test, y_pred_test)
print(f'Testing MSE: {mse_test:.2f}')

Validation MSE: 0.96
Testing MSE: 0.96


In [15]:
# Проведем K-fold валидацию с разбивкой базы на 10 частей

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create a KFold object with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Initialize lists to store the MSE values for each fold
mse_values = []

# Iterate over the folds
for train_index, val_index in kf.split(data.drop('target_log', axis=1)):
    # Split the data into training and validation sets for this fold
    X_train, X_val = data.drop('target_log', axis=1).iloc[train_index], data.drop('target_log', axis=1).iloc[val_index]
    y_train, y_val = data['target_log'].iloc[train_index], data['target_log'].iloc[val_index]

    # Train the model on the training data for this fold
    model.fit(X_train, y_train)

    # Make predictions on the validation data for this fold
    y_pred_val = model.predict(X_val)

    # Calculate the MSE for this fold
    mse = mean_squared_error(y_val, y_pred_val)

    # Append the MSE value to the list
    mse_values.append(mse)

# Calculate the average MSE across all folds
avg_mse = sum(mse_values) / len(mse_values)
print(f'Average MSE across all folds: {avg_mse:.2f}')

Average MSE across all folds: 0.96


In [9]:
data['target_log'].describe()

count    367043.000000
mean         12.642601
std           1.205926
min           0.000000
25%          12.128111
50%          12.676076
75%          13.286181
max          19.088510
Name: target_log, dtype: float64

In [16]:
print(f'Средняя ошибка модели составляет - {round(avg_mse**0.5/data['target_log'].mean()*100,4)} %')

Средняя ошибка модели составляет - 7.7622 %


**ВЫВОД**

Модели стохастического градиентного спуска и градиентного спуска для линейной регрессии дали плохие показатели и их расчет я здесь не привожу.

Предыдущие модели дали стабильную метрику - дающую ошибку в пределах 7,8 %. Я считаю, результат этой модели удовлетворительным.